In [1]:
import os
import xml.etree.ElementTree as ET

# Define the path to the XML file
file_path = "../Data/xmls_new/000C130.xml"  

# Function to recursively print the structure of the XML
def print_xml_structure(elem, level=0):
    indent = "  " * level
    print(f"{indent}Tag: {elem.tag}, Attributes: {elem.attrib}")
    for child in elem:
        print_xml_structure(child, level + 1)

# Function to parse the XML and print its structure
def study_xml_structure(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        print("Root Element:", root.tag)
        print("Structure:")
        print_xml_structure(root)
    except ET.ParseError as e:
        print(f"Error parsing {file_path}: {e}")

# Study the structure of the single XML file
study_xml_structure(file_path)


Root Element: rootTag
Structure:
Tag: rootTag, Attributes: {}
  Tag: Award, Attributes: {}
    Tag: AwardTitle, Attributes: {}
    Tag: AwardEffectiveDate, Attributes: {}
    Tag: AwardExpirationDate, Attributes: {}
    Tag: AwardAmount, Attributes: {}
    Tag: AwardInstrument, Attributes: {}
      Tag: Code, Attributes: {}
      Tag: Value, Attributes: {}
    Tag: Organization, Attributes: {}
      Tag: Code, Attributes: {}
      Tag: Directorate, Attributes: {}
        Tag: Abbreviation, Attributes: {}
        Tag: Code, Attributes: {}
        Tag: LongName, Attributes: {}
      Tag: Division, Attributes: {}
        Tag: Code, Attributes: {}
        Tag: Abbreviation, Attributes: {}
        Tag: LongName, Attributes: {}
    Tag: ProgramOfficer, Attributes: {}
      Tag: SignBlockName, Attributes: {}
    Tag: PropsalInformation, Attributes: {}
      Tag: ID, Attributes: {}
    Tag: AbstractNarration, Attributes: {}
    Tag: MinAmdLetterDate, Attributes: {}
    Tag: MaxAmdLetterDate, A

In [2]:
import pandas as pd
from datetime import datetime

def parse_xml(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()

        data = {
            'AwardID': root.findtext('.//AwardID'),
            'AwardTitle': root.findtext('.//AwardTitle'),
            'Agency': root.findtext('.//AGENCY'),
            'AwardEffectiveDate': root.findtext('.//AwardEffectiveDate'),
            'AwardExpirationDate': root.findtext('.//AwardExpirationDate'),
            'AwardTotalIntnAmount': root.findtext('.//AwardTotalIntnAmount'),
            'AwardAmount': root.findtext('.//AwardAmount'),
            'AwardInstrument': root.findtext('.//AwardInstrument/Value'),
            'Organization_Code': root.findtext('.//Organization/Code'),
            'Directorate_Abbreviation': root.findtext('.//Directorate/Abbreviation'),
            'Directorate_LongName': root.findtext('.//Directorate/LongName'),
            'Division_Abbreviation': root.findtext('.//Division/Abbreviation'),
            'Division_LongName': root.findtext('.//Division/LongName'),
            'ProgramOfficer_Name': root.findtext('.//ProgramOfficer/SignBlockName'),
            'ProgramOfficer_Email': root.findtext('.//ProgramOfficer/PO_EMAI'),
            'ProgramOfficer_Phone': root.findtext('.//ProgramOfficer/PO_PHON'),
            'AbstractNarration': root.findtext('.//AbstractNarration'),
            'MinAmdLetterDate': root.findtext('.//MinAmdLetterDate'),
            'MaxAmdLetterDate': root.findtext('.//MaxAmdLetterDate'),
            'TRAN_TYPE': root.findtext('.//TRAN_TYPE'),
            'CFDA_NUM': root.findtext('.//CFDA_NUM'),
            'NSF_PAR_USE_FLAG': root.findtext('.//NSF_PAR_USE_FLAG'),
            'FUND_AGCY_CODE': root.findtext('.//FUND_AGCY_CODE'),
            'AWDG_AGCY_CODE': root.findtext('.//AWDG_AGCY_CODE'),
            'Investigator1_Name': root.findtext('.//Investigator[1]/PI_FULL_NAME'),
            'Investigator1_Email': root.findtext('.//Investigator[1]/EmailAddress'),
            'Investigator2_Name': root.findtext('.//Investigator[2]/PI_FULL_NAME'),
            'Investigator2_Email': root.findtext('.//Investigator[2]/EmailAddress'),
            'Investigator3_Name': root.findtext('.//Investigator[3]/PI_FULL_NAME'),
            'Investigator3_Email': root.findtext('.//Investigator[3]/EmailAddress'),
            'Institution_Name': root.findtext('.//Institution/Name'),
            'Institution_City': root.findtext('.//Institution/CityName'),
            'Institution_State': root.findtext('.//Institution/StateName'),
            'Institution_Zip': root.findtext('.//Institution/ZipCode'),
            'Institution_Country': root.findtext('.//Institution/CountryName'),
            'Institution_Phone': root.findtext('.//Institution/PhoneNumber'),
            'Performance_Institution_Name': root.findtext('.//Performance_Institution/Name'),
            'Performance_Institution_City': root.findtext('.//Performance_Institution/CityName'),
            'Performance_Institution_State': root.findtext('.//Performance_Institution/StateName'),
            'Performance_Institution_Zip': root.findtext('.//Performance_Institution/ZipCode'),
            'Performance_Institution_Country': root.findtext('.//Performance_Institution/CountryName'),
            'ProgramElement1_Code': root.findtext('.//ProgramElement[1]/Code'),
            'ProgramElement1_Text': root.findtext('.//ProgramElement[1]/Text'),
            'ProgramElement2_Code': root.findtext('.//ProgramElement[2]/Code'),
            'ProgramElement2_Text': root.findtext('.//ProgramElement[2]/Text'),
            'ProgramElement3_Code': root.findtext('.//ProgramElement[3]/Code'),
            'ProgramElement3_Text': root.findtext('.//ProgramElement[3]/Text'),
            'ProgramReference_Code': root.findtext('.//ProgramReference/Code'),
            'ProgramReference_Text': root.findtext('.//ProgramReference/Text')
        }
        return data
    except ET.ParseError:
        print(f'Error parsing file: {file_path}')
        return None

def collect_data_from_xml_files(root_folder):
    data_list = []
    error_files = []

    for year_folder in os.listdir(root_folder):
        year_path = os.path.join(root_folder, year_folder)
        if os.path.isdir(year_path):  # Ensure it's a directory
            for filename in os.listdir(year_path):
                if filename.endswith('.xml'):
                    file_path = os.path.join(year_path, filename)
                    data = parse_xml(file_path)
                    if data is not None:
                        data['Year'] = year_folder
                        data_list.append(data)
                    else:
                        error_files.append(file_path)

    return pd.DataFrame(data_list), error_files

# Define the root folder where the year folders are located
root_folder = '../Data/unzipped_all'

# Collect data and create DataFrame
df, error_files = collect_data_from_xml_files(root_folder)

# Get the current date and time
now = datetime.now()
date_time_str = now.strftime("%Y%m%d_%H%M%S")

# Save the DataFrame to a CSV file with date and time in the filename
output_csv_path = f'output_data_{date_time_str}.csv'
df.to_csv(output_csv_path, index=False)

# Save the error log to a CSV file with date and time in the filename
error_log_path = f'error_log_{date_time_str}.csv'
pd.DataFrame(error_files, columns=['FilePath']).to_csv(error_log_path, index=False)

print(f'Data successfully saved to {output_csv_path}')
print(f'Error log saved to {error_log_path}')
print(f'Successfully parsed XML files: {len(df)}')
print(f'Files with errors: {len(error_files)}')

Error parsing file: ../Data/unzipped_all/2022/49100422C0020.xml
Error parsing file: ../Data/unzipped_all/2012/1201835.xml
Error parsing file: ../Data/unzipped_all/1975/7508378.xml
Error parsing file: ../Data/unzipped_all/1987/8711837.xml
Error parsing file: ../Data/unzipped_all/1987/8716860.xml
Error parsing file: ../Data/unzipped_all/1987/8710142.xml
Error parsing file: ../Data/unzipped_all/1993/9318513.xml
Error parsing file: ../Data/unzipped_all/1993/9315882.xml
Error parsing file: ../Data/unzipped_all/1993/9314569.xml
Error parsing file: ../Data/unzipped_all/1992/9214257.xml
Error parsing file: ../Data/unzipped_all/2008/0834451.xml
Error parsing file: ../Data/unzipped_all/2008/0848615.xml
Error parsing file: ../Data/unzipped_all/2008/0833162.xml
Error parsing file: ../Data/unzipped_all/2008/0810982.xml
Error parsing file: ../Data/unzipped_all/2008/0803440.xml
Error parsing file: ../Data/unzipped_all/2008/0831791.xml
Error parsing file: ../Data/unzipped_all/2008/0826158.xml
Error pa